In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

MODEL_URL = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
detector = hub.load(MODEL_URL).signatures['default']

def detect_objects_in_video(video_path, output_path=None):
    cap = cv2.VideoCapture(video_path)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    if output_path:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_rgb, (512, 512)).astype(np.float32) / 255.0
        img_tensor = tf.convert_to_tensor([img_resized], dtype=tf.float32)

        detections = detector(img_tensor)

        detection_boxes = detections["detection_boxes"].numpy()
        detection_classes = detections["detection_class_entities"].numpy()
        detection_scores = detections["detection_scores"].numpy()

        for i in range(len(detection_scores)):
            if detection_scores[i] >= 0.6:
                box = detection_boxes[i]
                y1, x1, y2, x2 = (
                    int(box[0] * frame.shape[0]),
                    int(box[1] * frame.shape[1]),
                    int(box[2] * frame.shape[0]),
                    int(box[3] * frame.shape[1]),
                )
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                class_name = detection_classes[i].decode("utf-8")
                label = f"{class_name}: {detection_scores[i] * 100:.0f}%"
                cv2.putText(
                    frame,
                    label,
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (0, 255, 0),
                    2,
                    cv2.LINE_AA,
                )

        if output_path:
            out.write(frame)

    cap.release()
    if output_path:
        out.release()
        print("Video processing complete. The output video was saved successfully.")
    else:
        print("Video processing complete.")

video_path = "daivik.mp4"
output_path = "C:/Users/priya/Downloads/new project ml/sample_v.mp4"
detect_objects_in_video(video_path, output_path)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Video processing complete. The output video was saved successfully.


In [8]:

pip install opencv-python


  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\priya\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.



In [18]:
import cv2
print(cv2.__version__)


4.10.0
